In [1]:
import requests
from bs4 import BeautifulSoup
import csv
import pandas as pd
import re
import datetime
import ftfy

In [3]:
#read in city CSV to get list of links
#Please note that though the encoding is nominally UTF-8, 
#CSV really doesn't like it and outputs mojibake instead of proper accents

hostellinks = pd.read_csv('hostelinks20180924-211021.csv', header=0, encoding="utf-8")
#hostellinks.to_sql('hostel_list', engine, if_exists = 'replace')

In [4]:
#get today's date
now = datetime.datetime.today()

#we set up a time that's 90 days in the past (3 months), just to pick up a few more reviews
then = now + datetime.timedelta(-90)

In [7]:
import time
now = time.strftime("%Y%m%d-%H%M%S")
csvname = 'hostelreviews-id'+now+'.csv'

In [ ]:
#hostelworld displays 10 reviews to a page. Now we start scraping and processing raw data

In [8]:
def hosteldujour(index):
    n = ftfy.fix_text(hostellinks['name'][index])
    i = hostellinks['hostelid'][index]
    c = hostellinks['city'][index]
    u = hostellinks['url'][index] + '/reviews/'
    return n,i,c,u

In [10]:
with open(csvname, 'w', newline='', encoding="UTF-8") as f:
    writer = csv.writer(f)
    writer.writerow(['name', 'hostelid','city','date', 'reviewid','reviewtext','value','security','location','facilities','staff','atmosphere','cleanliness'])
    
    for ind in hostellinks.index:
        loopbreaker = 0
        hostelname, hostelid, hostelcity, hostelurl = hosteldujour(ind)

        if ind == 0:
            review_page_url = hostelurl + '?showOlderReviews=1&page=1'
            response = requests.get(review_page_url)
            hostel_soup = BeautifulSoup(response.text, 'html.parser')
        
            if not hostel_soup.find(class_="rating-no-rating"):
                last_page = hostel_soup.find(class_="reviews-count")
                lastnum = str(last_page).split(" ")[4] #this is the hackiest solution eve
            else:
                continue;
                
        for i in range(1,int(lastnum)):
            review_page_url = hostelurl + '?showOlderReviews=1&page=' + str(i)
            print(review_page_url)
                
            response = requests.get(review_page_url)
            hostel_soup = BeautifulSoup(response.text, 'html.parser')
                
            page_reviews = hostel_soup.find_all(class_="reviewdetails")
            if not page_reviews:
                break
            else:
                for page_review_components in page_reviews:
                    #fix date
                    tempdate = page_review_components.find(class_="reviewdate")
                    temp = str(tempdate).split(" ")
                    temp = re.sub("[^0-9]", "", temp[4]) + " " + temp[5] + " " + temp[6][:4]
                    date = datetime.datetime.strptime(temp, '%d %b %Y')

                    if date > then:
                        #fix review text
                        rtext = page_review_components.find(class_="reviewtext")
                        reviewid = rtext.get('id')
                        temp = str(rtext).split("p>")
                        t = ftfy.fix_text(temp[1][:-2])

                        #fix ratings
                        temprate = page_review_components.find_all(class_="ratinglist")
                        temp = str(temprate).split("<span>")
                        val=int(temp[1].split(".")[0])
                        sec=int(temp[2][:2].split(".")[0])
                        loc=int(temp[3][:2].split(".")[0])
                        fac=int(temp[4][:2].split(".")[0])
                        staff=int(temp[5][:2].split(".")[0])
                        atmo=int(temp[6][:2].split(".")[0])
                        cl=int(temp[7][:2].split(".")[0])

                        writer.writerow( [ hostelname, hostelid, hostelcity, date, reviewid, t, val, sec, loc, fac, staff, atmo, cl ] )
                    else:
                        loopbreaker = 1
                        break
                    
            if loopbreaker > 0:
                break

https://www.hostelworld.com/hosteldetails.php/MEININGER-Berlin-East-Side-Gallery/Berlin/280374/reviews/?showOlderReviews=1&page=1
https://www.hostelworld.com/hosteldetails.php/MEININGER-Berlin-East-Side-Gallery/Berlin/280374/reviews/?showOlderReviews=1&page=2
https://www.hostelworld.com/hosteldetails.php/MEININGER-Berlin-East-Side-Gallery/Berlin/280374/reviews/?showOlderReviews=1&page=3
https://www.hostelworld.com/hosteldetails.php/MEININGER-Berlin-East-Side-Gallery/Berlin/280374/reviews/?showOlderReviews=1&page=4
https://www.hostelworld.com/hosteldetails.php/MEININGER-Berlin-East-Side-Gallery/Berlin/280374/reviews/?showOlderReviews=1&page=5
https://www.hostelworld.com/hosteldetails.php/Grand-Hostel-Berlin-Classic/Berlin/34160/reviews/?showOlderReviews=1&page=1
https://www.hostelworld.com/hosteldetails.php/Grand-Hostel-Berlin-Classic/Berlin/34160/reviews/?showOlderReviews=1&page=2
https://www.hostelworld.com/hosteldetails.php/Grand-Hostel-Berlin-Classic/Berlin/34160/reviews/?showOlderR

In [11]:
del writer